# Agent Template

from langchain_community.tools.tavily_search import TavilySearchResults ## tavily search

from langchain.tools.retriever import create_retriever_tool ## tool + retriever

`create_retriever_tool` 안에 있는 description이 agent에게 역할을 주는것이라고 생각하면 된다.

from langchain.agents import create_openai_functions_agent, AgentExecutor ## agent 

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_teddynote import logging
logging.langsmith("CH-15 Agent_template")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH-15 Agent_template


In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults ## tavily search
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.tools.retriever import create_retriever_tool ## tool + retriever
from langchain import hub
from langchain.agents import create_openai_functions_agent, AgentExecutor ## agent

# memory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [4]:
# 1.search 도구
search = TavilySearchResults(k=5)

# 1-2.indexing
loader = PyPDFLoader("./data/SPRI_AI_Brief_2023년12월호_F.pdf")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

documents = text_splitter.split_documents(docs)

embeddings = OpenAIEmbeddings(model='text-embedding-3-small')
vector_store = FAISS.from_documents(documents, embeddings)
retriever = vector_store.as_retriever()

# 1-3.tool모듈에서 retriever도구 생성 및 도구 정의
retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",
    description="2023년 12월 AI 관련 정보를 PDF 문서에서 검색합니다. '2023년 12월 AI 산업동향' 과 관련된 질문은 이 도구를 사용해야 합니다!",
)

tools = [search, retriever_tool]

In [5]:
# 2.llm 정의
llm = ChatOpenAI(model="gpt-4o-mini")

In [12]:
# 3.prompt 정의
# prompt = hub.pull("hwchase17/openai-functions-agent")
# prompt.pretty_print()
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a helpful assistant"),
  ("placeholder", "{chat_history}"),
  ("human", "{input}"),
  ("placeholder", "{agent_scratchpad}"),
])
prompt.pretty_print()

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


In [7]:
# 4.Agent정의 : 위에서 정의한 llm, toos, prompt를 인자로 사용한다.
agent = create_openai_functions_agent(llm, tools, prompt)

In [8]:
# 5.Agent Excutor 정의
# AgentExecutor 클래스를 사용하여 agent와 tools를 설정하고, 상세한 로그를 출력하도록 verbose를 True로 설정합니다.
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [9]:
# 6.채팅기록을 정의하는 메모리 추가
message_history = ChatMessageHistory()

In [10]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history"
)

In [11]:
# 7. 질의응답 테스트
response = agent_with_chat_history.invoke(
    {
        "input": "YouTube 2024년부터 AI 생성콘텐츠 표시 의무화에 대한 내용을 PDF 문서에서 알려줘"
    },
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "MyTestSessionID"}},
)
print(f"답변: {response['output']}")



> Entering new AgentExecutor chain...

Invoking: `pdf_search` with `{'query': 'YouTube 2024년 AI 생성콘텐츠 표시 의무화'}`


SPRi AI Brief |  
2023-12 월호
14유튜브 , 2024년부터 AI 생성 콘텐츠 표시 의무화 
n유튜브가 몇 달 안에 생성 AI를 사용한 콘텐츠에 AI 라벨 표시를 의무화하기로 했으며 , 이를 
준수하지 않는 콘텐츠는 삭제하고 크리에이터에 대한 수익 배분도 중단할 수 있다고 설명
n유튜브는 AI 생성 콘텐츠가 신원 파악이 가능한 개인을 모방한 경우 개인정보 침해 신고 
절차에 따라 콘텐츠 삭제 요청도 받을 계획KEY Contents
£유튜브 , 생성 AI 콘텐츠에 AI 라벨 표시 안 하면 콘텐츠 삭제
n유튜브가 2023 년 11월 14일 공식 블로그를 통해 몇 달 안에 생성 AI를 사용한 콘텐츠에 AI 
라벨을 표시하는 새로운 규칙을 시행한다고 발표 
∙실제로 일어나지 않은 사건을 사실적으로 묘사하거나 실제로 하지 않은 말이나 행동을 보여주는 콘텐츠와 
같이 AI 도구를 사용해 사실적으로 변경되거나 합성된 콘텐츠에는 AI 라벨을 표시 필요
∙유튜브는 이러한 규칙이 선거나 분쟁 상황, 공중 보건, 공직자 관련 문제와 같이 민감한 주제를 다루는 
콘텐츠에서 특히 중요하다고 강조했으며 , 크리에이터가 AI로 제작한 콘텐츠에 AI 라벨을 표시하지 않으면 
해당 콘텐츠는 삭제되고 광고 수익을 배분하는 유튜브 파트너 프로그램도 정지될 수 있음
∙유튜브는 두 가지 방식으로 AI를 이용한 콘텐츠의 변경이나 합성 여부를 시청자에게 전달할 계획으로 
동영상 설명 패널에 라벨을 표시하는 방식이 기본이며 , 민감한 주제를 다루는 특정 유형의 콘텐츠는 동영상 
플레이어에 더욱 눈에 띄는 라벨을 적용 
∙유튜브는 커뮤니티 정책에 위반되는 일부 합성 콘텐츠에 대해서는 라벨 지정 여부와 관계없이 삭제할 
방침으로 , 가령 사실적인 폭력을 보여주는 합성 동영상이 시청자